# Tutorial 4: Schema without Chain of Thought (CoT)

In this tutorial, we will explore how to use RadPrompter to work with schemas without using Chain of Thought (CoT).

So let's get started!

## 0. Imports

In [3]:
import os
import pandas as pd
import glob
from radprompter import RadPrompter, Prompt, vLLMClient, OllamaClient

## 1. Load reports

In [4]:
report_files = glob.glob("../../sample_reports/*.txt")

In [5]:
reports = []
report_ids = []
for report_file in report_files:
    report_ids.append(os.path.basename(report_file))
    with open(report_file, 'r') as f:
        reports.append(f.read())

# 2. Initializing Prompt

In [6]:
prompt = Prompt('4_Using-Schemas.toml')

In [7]:
prompt

We can look at the each schema by passing the index `i`: `prompt.schema[i]`

In this example, we see that the first schema contains the introduction prompt which contains the Radiology report.

It also contains the constraints for the LLM and the first question.

In [8]:
prompt.schemas[0]

Subsequent schemas directly go to the question and hints, as shown below.

This is because these schemas are not independent from the previous schema.

In [9]:
prompt.schemas[1]

## 3. Initializing LLM Client and RadPrompter Engine

In [10]:
client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

In [11]:
engine = RadPrompter(
    client=client,
    prompt=prompt,
    concurrency=12,
    hide_blocks=False,
    output_file="output_tutorial_4.csv",
)

## 4. Run Inference

In [12]:
sample = [{'report': sample_report, 'report_id': report_id} for sample_report, report_id in zip(reports, report_ids)]

In [13]:
engine(sample)

Processing items:   0%|          | 0/3 [00:00<?, ?it/s]

Processing items: 100%|██████████| 3/3 [00:01<00:00,  1.80it/s]


In [14]:
results = pd.read_csv("output_tutorial_4.csv", index_col=0)
results

,Pulmonary Embolism_response,Left_response,Right_response,Acute_response,Chronic_response,RightHeartStrain_response,PulmonaryArteryHypertension_response,report,report_id
index,,,,,,,,,
0,Present,Yes,Yes,No,Yes,No,Yes,Clinical Information:\n67-year-old male with s...,sample_report_1.txt
2,Present,Yes,Yes,No,No,No,No,Here is an example radiology report describing...,sample_report_3.txt
1,Present,Yes,Yes,Yes,No,Yes,No,Clinical Information:\n72-year-old female with...,sample_report_2.txt


In [15]:
engine.save_log("log_tutorial_4.log")